In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, when, isnan, isnull
from pyspark.sql.types import IntegerType, StringType, FloatType
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [ ]:
spark = SparkSession.builder.appName(
            "SentimentAnalysisBaseline"
        ).config(
            "spark.executor.memory", "16g"
        ).config(
            "spark.driver.memory", "32g"
        ).getOrCreate()

In [ ]:
df = spark.read.csv("../ggg_sg.csv", header=True, inferSchema=True, multiLine=True, escape='"')

In [ ]:
df.show()

In [ ]:
df = df.filter(df.ContextualText.isNotNull())
df = df.filter(df.DocTone.isNotNull())

In [ ]:
df = df.withColumn("DocTone", df["DocTone"].cast(FloatType()))

In [ ]:
df.show(10)

# Exploratory Data Analysis (DocTone)

In [ ]:
from pyspark.sql.functions import col, min, max, avg, stddev, percentile_approx

stats = df.select(
    min(col("DocTone")).alias("min_DocTone"),
    max(col("DocTone")).alias("max_DocTone"),
    avg(col("DocTone")).alias("avg_DocTone"),
    stddev(col("DocTone")).alias("stddev_DocTone")
).collect()[0]

print("DocTone Statistics:")
print(f"Min: {stats['min_DocTone']}")
print(f"Max: {stats['max_DocTone']}")
print(f"Average: {stats['avg_DocTone']}")
print(f"Standard Deviation: {stats['stddev_DocTone']}")


In [ ]:
# Calculate quantiles
quantiles = [0.0, 0.25, 0.5, 0.75, 1.0]

# Calculate quantiles using approxQuantile
quantile_values = df.approxQuantile("DocTone", quantiles, relativeError=0.001)

# Print quantiles
for q, value in zip(quantiles, quantile_values):
    print(f"{int(q*100)}th percentile: {value}")


In [ ]:
# Sample data for visualization (100,000 samples)
sample_df = df.select("DocTone").sample(False, 0.01, seed=42).limit(100000)

# Collect DocTone values
doc_tone_values = sample_df.rdd.map(lambda row: row['DocTone']).collect()

import matplotlib.pyplot as plt
import seaborn as sns

# Plot distribution of DocTone scores
plt.figure(figsize=(10, 6))
sns.histplot(doc_tone_values, bins=50, kde=True)
plt.title('Distribution of DocTone Scores (Sampled Data)')
plt.xlabel('DocTone Score')
plt.ylabel('Frequency')
plt.show()


DocTone Statistics:

Min: -28.947368621826172

Max: 34.78260803222656

Average: -0.03639324925228623

Standard Deviation: 3.1345220942204715

0th percentile: -28.947368621826172

25th percentile: -2.0202019214630127

50th percentile: 0.0

75th percentile: 1.9910084009170532

100th percentile: 34.78260803222656

# TF-IDF + Logistic Regression

In [ ]:
# Create sentiment label: Positive (2), Neutral (1), Negative (0)
def sentiment_label(score):
    if score > 1.9910:
        return 2
    elif score < -2.0202:
        return 0
    else:
        return 1

sentiment_udf = udf(sentiment_label, IntegerType())

df = df.withColumn("label", sentiment_udf(col("DocTone")))

label_counts = df.groupBy("label").count().orderBy("label")
label_counts.show()

In [ ]:
# Tokenize
tokenizer = Tokenizer(inputCol="ContextualText", outputCol="words")

# Remove stop words
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")

# Feature extraction (TF-IDF)
hashingTF = HashingTF(inputCol="filtered_words", outputCol="rawFeatures", numFeatures=10000)
idf = IDF(inputCol="rawFeatures", outputCol="features")

pipeline = Pipeline(stages=[tokenizer, remover, hashingTF, idf])

In [ ]:
(trainingData, testData) = df.randomSplit([0.8, 0.2], seed=42)

pipelineModel = pipeline.fit(trainingData)
trainingData = pipelineModel.transform(trainingData)
testData = pipelineModel.transform(testData)

# Train logistic regression model
lr = LogisticRegression(featuresCol='features', labelCol='label', maxIter=10, family='multinomial')
lrModel = lr.fit(trainingData)


In [ ]:
# Make predictions
predictions = lrModel.transform(testData)

In [ ]:
# Evaluate model
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Accuracy = %g " % accuracy)

In [ ]:
# Detailed evaluation
evaluator_precision = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
precision = evaluator_precision.evaluate(predictions)

evaluator_recall = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")
recall = evaluator_recall.evaluate(predictions)

evaluator_f1 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
f1 = evaluator_f1.evaluate(predictions)

print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# Word2Vec + Random Forest

In [ ]:
# from pyspark.ml.feature import Word2Vec
# from pyspark.ml.classification import RandomForestClassifier
#
# # Create sentiment label: Positive (2), Neutral (1), Negative (0)
# def sentiment_label(score):
#     if score > 1.9910:
#         return 2
#     elif score < -2.0202:
#         return 0
#     else:
#         return 1
#
# sentiment_udf = udf(sentiment_label, IntegerType())
#
# df = df.withColumn("label", sentiment_udf(col("DocTone")))
# tokenizer = Tokenizer(inputCol="ContextualText", outputCol="words")
# remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
# # Feature extraction (Word2Vec)
# word2Vec = Word2Vec(vectorSize=100, minCount=5, inputCol="filtered_words", outputCol="features")
#
# pipeline = Pipeline(stages=[tokenizer, remover, word2Vec])
# (trainingData, testData) = df.randomSplit([0.8, 0.2], seed=42)
#
# pipelineModel = pipeline.fit(trainingData)
# trainingData = pipelineModel.transform(trainingData)
# testData = pipelineModel.transform(testData)
#
# # Random Forest model
# rf = RandomForestClassifier(featuresCol='features', labelCol='label', numTrees=50, maxDepth=10, seed=42)
#
# rfModel = rf.fit(trainingData)

In [ ]:
# predictions = rfModel.transform(testData)

In [ ]:
# # Evaluate model
# evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
# accuracy = evaluator.evaluate(predictions)
# print("Test Accuracy = %g " % accuracy)
#
# # Detailed evaluation
# evaluator_precision = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedPrecision")
# precision = evaluator_precision.evaluate(predictions)
#
# evaluator_recall = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="weightedRecall")
# recall = evaluator_recall.evaluate(predictions)
#
# evaluator_f1 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="f1")
# f1 = evaluator_f1.evaluate(predictions)
#
# print(f"Precision: {precision}")
# print(f"Recall: {recall}")
# print(f"F1 Score: {f1}")
#
# predictions.select("ContextualText", "label", "prediction").show(10, truncate=50)